In [1]:
import pandas as pd
import numpy as np
import swifter

In [2]:
airlines = pd.read_csv('data/airlines.csv')
airports = pd.read_csv('data/airports.csv')

In [3]:
flights = pd.read_csv('data/flights.csv', low_memory=False)

In [4]:
display(airlines.head(6))
display(airports.head(7))
display(flights.head(6))

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.


,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
5,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
6,ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
flights['has_origin']= flights['ORIGIN_AIRPORT'].apply(lambda x: not x.isdecimal())

In [5]:
flights_active = flights[(flights['CANCELLED']==0 )& (flights['DIVERTED']==0) & (flights['has_origin']==True)]

In [6]:
print(flights.shape[0])
print(flights_active.shape[0])
del flights

5819079
5231130


In [7]:
## convert flight schedule to datetime - todelete
flights_ac_sch_dep = pd.DataFrame(columns = ['hour','minute'])
flights_ac_sch_dep['hour'] = flights_active['SCHEDULED_DEPARTURE']//100
flights_ac_sch_dep['minute'] = flights_active['SCHEDULED_DEPARTURE']%100
scheduled_flights = pd.to_datetime({'year':flights_active['YEAR'], 'month':flights_active['MONTH'], 'day':flights_active['DAY'], 'hour':flights_ac_sch_dep['hour'], 'minute': flights_ac_sch_dep['minute']})
flights_active.loc[:,'scheduled_datetime'] = scheduled_flights[flights_active.index]
del scheduled_flights

In [8]:
# add departure datetime
departure_datetime = flights_active['scheduled_datetime'] + pd.to_timedelta(flights_active['DEPARTURE_DELAY'],unit='m')
flights_active['departure_datetime'] = departure_datetime[flights_active.index]
del departure_datetime

In [24]:
flights_active[['MONTH','DAY','SCHEDULED_DEPARTURE','scheduled_datetime','DEPARTURE_DELAY','departure_datetime']]

,MONTH,DAY,SCHEDULED_DEPARTURE,scheduled_datetime,DEPARTURE_DELAY,departure_datetime
0,1,1,5,2015-01-01 00:05:00,-11.0,2014-12-31 23:54:00-09:00
1,1,1,10,2015-01-01 00:10:00,-8.0,2015-01-01 00:02:00-08:00
2,1,1,20,2015-01-01 00:20:00,-2.0,2015-01-01 00:18:00-08:00
3,1,1,20,2015-01-01 00:20:00,-5.0,2015-01-01 00:15:00-08:00
4,1,1,25,2015-01-01 00:25:00,-1.0,2015-01-01 00:24:00-08:00
...,...,...,...,...,...,...
5819074,12,31,2359,2015-12-31 23:59:00,-4.0,2015-12-31 23:55:00-08:00
5819075,12,31,2359,2015-12-31 23:59:00,-4.0,2015-12-31 23:55:00-05:00
5819076,12,31,2359,2015-12-31 23:59:00,-9.0,2015-12-31 23:50:00-05:00
5819077,12,31,2359,2015-12-31 23:59:00,-6.0,2015-12-31 23:53:00-05:00


In [9]:
# add arival datetime
arrival_datetime = flights_active['departure_datetime'] + pd.to_timedelta(flights_active['ELAPSED_TIME'],unit='m')
flights_active['arrival_datetime'] = arrival_datetime[flights_active.index]

In [10]:
# delete flights from days when day saving date change
# fall daylight saving time
excluded = flights_active[( (flights_active['MONTH']==9) & (flights_active['DAY']==31) ) | ( (flights_active['MONTH']==11) & (flights_active['DAY']==1) )]
# spring daylight saving time
excluded = excluded.append(flights_active[( (flights_active['MONTH']==3) & (flights_active['DAY']==7) ) | ( (flights_active['MONTH']==3) & (flights_active['DAY']==8) )])
flights_active = flights_active.append(excluded).drop_duplicates(keep=False)

In [11]:
flights_active[(flights_active['MONTH']==11)&(flights_active['DAY']==1)]

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,has_origin,scheduled_datetime,departure_datetime,arrival_datetime


In [12]:
airports[airports.isna().any(axis=1)]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [13]:
# Fill empty coordinates
airports.loc[airports['IATA_CODE'] == 'ECP', ['LATITUDE']] = 30.35521
airports.loc[airports['IATA_CODE'] == 'ECP', ['LONGITUDE']] = -85.79956
airports.loc[airports['IATA_CODE'] == 'PBG', ['LATITUDE']] = 44.65349
airports.loc[airports['IATA_CODE'] == 'PBG', ['LONGITUDE']] = -73.46792
airports.loc[airports['IATA_CODE'] == 'UST', ['LATITUDE']] = 29.95518
airports.loc[airports['IATA_CODE'] == 'UST', ['LONGITUDE']] = -81.34300
airports.loc[[96,234,313],:]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,30.35521,-85.79956
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,44.65349,-73.46792
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,29.95518,-81.34300


In [14]:
# add timezones to airports
from timezonefinder import TimezoneFinder

finder = TimezoneFinder().certain_timezone_at 
airports_timezones = airports.apply(lambda x: finder(lng=x['LONGITUDE'], lat=x['LATITUDE']),axis=1)
airports['timezone'] = airports_timezones[airports_timezones.index]

In [15]:
airports.loc[airports.IATA_CODE == "PPG", "timezone"] = "Pacific/Samoa"
airports.loc[airports.IATA_CODE == "GUM", "timezone"] = "Pacific/Guam"

In [16]:
airports[airports['IATA_CODE']=='GUM']

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,timezone
137,GUM,Guam International Airport,Agana,GU,USA,13.48345,-144.79598,Pacific/Guam


In [17]:
# create origin timezone
flights_active['origin_timezone'] = flights_active.join(airports.set_index('IATA_CODE'), on='ORIGIN_AIRPORT', how='left')['timezone']
flights_active.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,has_origin,scheduled_datetime,departure_datetime,arrival_datetime,origin_timezone
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:05:00,2014-12-31 23:54:00,2015-01-01 03:08:00,America/Anchorage
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:10:00,2015-01-01 00:02:00,2015-01-01 04:41:00,America/Los_Angeles
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:20:00,2015-01-01 00:18:00,2015-01-01 05:11:00,America/Los_Angeles
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:20:00,2015-01-01 00:15:00,2015-01-01 04:56:00,America/Los_Angeles
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:25:00,2015-01-01 00:24:00,2015-01-01 03:59:00,America/Los_Angeles


In [18]:
# create desitnation timezone
flights_active['destination_timezone'] = flights_active.join(airports.set_index('IATA_CODE'), on='DESTINATION_AIRPORT', how='left')['timezone']
flights_active.tail(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,has_origin,scheduled_datetime,departure_datetime,arrival_datetime,origin_timezone,destination_timezone
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,NaN,NaN,NaN,NaN,True,2015-12-31 23:59:00,2015-12-31 23:55:00,2016-01-01 04:53:00,America/Los_Angeles,America/New_York
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,NaN,NaN,NaN,NaN,True,2015-12-31 23:59:00,2015-12-31 23:55:00,2016-01-01 03:30:00,America/New_York,America/Puerto_Rico
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,NaN,NaN,NaN,NaN,True,2015-12-31 23:59:00,2015-12-31 23:50:00,2016-01-01 03:32:00,America/New_York,America/Puerto_Rico
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,NaN,NaN,NaN,NaN,True,2015-12-31 23:59:00,2015-12-31 23:53:00,2016-01-01 02:30:00,America/New_York,America/Puerto_Rico
5819078,2015,12,31,4,B6,839,N534JB,JFK,BQN,2359,...,NaN,NaN,NaN,NaN,True,2015-12-31 23:59:00,2016-01-01 00:14:00,2016-01-01 03:42:00,America/New_York,America/Puerto_Rico


In [ ]:
# lacking timezones
print(flights_active[flights_active['origin_timezone'].isna()]['ORIGIN_AIRPORT'].drop_duplicates())
print(flights_active[flights_active['destination_timezone'].isna()]['DESTINATION_AIRPORT'].drop_duplicates())

In [19]:
#add timezone todeparture_daterime
from tqdm import tqdm
tqdm.pandas()
departure_datetime_timezone = flights_active.progress_apply(lambda x: x['departure_datetime'].tz_localize(tz=x['origin_timezone']),axis=1)
departure_datetime_timezone

100%|██████████| 5185537/5185537 [08:22<00:00, 10320.44it/s]


0          2014-12-31 23:54:00-09:00
1          2015-01-01 00:02:00-08:00
2          2015-01-01 00:18:00-08:00
3          2015-01-01 00:15:00-08:00
4          2015-01-01 00:24:00-08:00
                     ...            
5819074    2015-12-31 23:55:00-08:00
5819075    2015-12-31 23:55:00-05:00
5819076    2015-12-31 23:50:00-05:00
5819077    2015-12-31 23:53:00-05:00
5819078    2016-01-01 00:14:00-05:00
Length: 5185537, dtype: object

In [20]:
flights_active['departure_datetime'] = departure_datetime_timezone[departure_datetime_timezone.index]
del departure_datetime_timezone

In [21]:
# original arrival_time is set to origin_timezone (arrival time was calculated basing on departure time)
flights_active['arrival_datetime'] = flights_active.progress_apply(lambda x: x['arrival_datetime'].tz_localize(tz=x['origin_timezone']),axis=1)

100%|██████████| 5185537/5185537 [08:07<00:00, 10627.17it/s]


In [23]:
#set arrival timezone to destination_timezone
#flights_active['arrival_datetime'] = flights_active.progress_apply(lambda x: x['arrival_datetime'].tz_convert(tz=x['destination_timezone']), axis=1)
flights_active['arrival_datetime'] = flights_active.progress_apply(lambda x: x['arrival_datetime'].tz_convert(tz=x['destination_timezone']), axis=1)

100%|██████████| 5185537/5185537 [04:09<00:00, 20798.63it/s]


In [25]:
flights_active.to_pickle("./data/flights_fixed.pkl")
flights_active.to_csv("./data/flights_fixed.csv", index=False)

airports.to_pickle("./data/airports_fixed.pkl")
airports.to_csv("./data/airports_fixed.csv", index=False)

In [ ]:
# utworzyć joina z wspórzędnymi i datami wylotu i przylotu
# Przeliczyć datetime na tylko datę
#

#location = flights_active.join(airports.set_index('IATA_CODE'), on='ORIGIN_AIRPORT', how='left')

In [40]:
# create location-hour
departure_location = flights_active['ORIGIN_AIRPORT']
departure_hours = flights_active.progress_apply(lambda x: x['departure_datetime'].round(freq='H'), axis=1)
departure_hours_location = pd.DataFrame({"airport":departure_location, "time":departure_hours})
len(departure_hours_location)

100%|██████████| 5185537/5185537 [22:57<00:00, 3764.53it/s] 


5185537

In [46]:
target_location = flights_active['DESTINATION_AIRPORT']
target_hours = flights_active.progress_apply(lambda x: x['arrival_datetime'].round(freq='H'), axis=1)
target_hours_location = pd.DataFrame({"airport":target_location, "time":target_hours})
len(target_hours_location)

100%|██████████| 5185537/5185537 [20:19<00:00, 4253.48it/s] 


5185537

In [47]:
hours_location = pd.concat([departure_hours_location,target_hours_location], ignore_index=True)
del departure_hours_location
del target_hours_location
len(hours_location)

10371074

In [48]:
hours_location = hours_location.groupby(hours_location.columns.tolist(),as_index=False).size()
len(hours_location)

1176467

In [49]:
hours_location[['longitude','latitude']] = hours_location.join(airports.set_index('IATA_CODE'), on='airport', how='left')[['LATITUDE','LONGITUDE']]
hours_location

,airport,time,size,longitude,latitude
0,ABE,2015-01-01 02:00:00-09:00,2,40.65236,-75.44040
1,ABE,2015-01-01 14:00:00-06:00,1,40.65236,-75.44040
2,ABE,2015-01-01 13:00:00-08:00,1,40.65236,-75.44040
3,ABE,2015-01-01 12:00:00-10:00,1,40.65236,-75.44040
4,ABE,2015-01-01 18:00:00-08:00,1,40.65236,-75.44040
...,...,...,...,...,...
1176462,YUM,2015-12-31 13:00:00-07:00,1,32.65658,-114.60597
1176463,YUM,2015-12-31 15:00:00-08:00,2,32.65658,-114.60597
1176464,YUM,2015-12-31 20:00:00-05:00,1,32.65658,-114.60597
1176465,YUM,2015-12-31 18:00:00-08:00,1,32.65658,-114.60597


In [50]:
hours_location.to_pickle("./hours_location.pkl")
hours_location.to_csv("./hours_location.csv", index=False)

In [24]:
pd.set_option("display.max_columns", None)
flights_active.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,has_origin,scheduled_datetime,departure_datetime,arrival_datetime,origin_timezone,destination_timezone,flight_time
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:05:00,2014-12-31 23:54:00-09:00,2015-01-01 03:08:00-09:00,America/Anchorage,America/Los_Angeles,194.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:10:00,2015-01-01 00:02:00-08:00,2015-01-01 04:41:00-08:00,America/Los_Angeles,America/New_York,279.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:20:00,2015-01-01 00:18:00-08:00,2015-01-01 05:11:00-08:00,America/Los_Angeles,America/New_York,293.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:20:00,2015-01-01 00:15:00-08:00,2015-01-01 04:56:00-08:00,America/Los_Angeles,America/New_York,281.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True,2015-01-01 00:25:00,2015-01-01 00:24:00-08:00,2015-01-01 03:59:00-08:00,America/Los_Angeles,America/Anchorage,215.0
